## # Preprocessing.ipynb

### # from Raw to Preprocessed_for_classification

In [1]:
import sys
import os
import os.path as osp
import re
from glob import glob
import shutil

import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
base_PATH='../Dataset/raw/classification'
# 전체 데이터 경로 가져오기
total_path=glob(osp.join(base_PATH, '*', '*.jpg'))

print('전체 데이터 수:',len(total_path))

전체 데이터 수: 30423


In [3]:
## csv 파일에서 달걀 정보 추출
#df_info=pd.read_csv('label_info.csv')

In [4]:
#df_full=pd.DataFrame(columns=['day','egg','id','label','image_path','json_path'])
df_full=pd.DataFrame(columns=['name','label','image_path','json_path'])

for path in tqdm(total_path):

    # 이미지 이름 추출
    image_name=osp.split(path)[-1]
    # 해당 데이터의 정보 추출
    #info=df_info[df_info['image_name']==image_name]
    
    
    # json 경로 생성
    json_path=path.replace('.jpg', '.json')
    # 이미지 경로에서 label 추출
    #label=info['label'].item()
    label= path.split(os.path.sep)[-2]
    ## 이미지 이름에서 day 추출
    #day=info['date'].item()
    ## 이미지 이름에서 id 추출
    #id_=info['id'].item()
    ## id 에서 egg 추출
    #egg=info['egg'].item()
    
    

    # label과 image_path 의 dictionary 생성
    #dict_={'day':day, 'egg':egg, 'id':id_, 'label': label, 'image_path': path, 'json_path':json_path}
    dict_={'name':image_name, 'label': label, 'image_path': path, 'json_path':json_path}
    df_full=df_full.append(dict_, ignore_index=True)

100%|██████████| 30423/30423 [01:18<00:00, 387.55it/s]


In [5]:
# train, validation, test set 분할  8 : 1 : 1
#unique_df=df_full.drop_duplicates(subset=['day','egg'])
unique_df=df_full.drop_duplicates(subset=['name'])
train_val_id, test_df, _,_=train_test_split(unique_df, unique_df['label'], test_size=0.1, stratify=unique_df['label'], random_state=42)
train_id, val_df,_,_=train_test_split(train_val_id, train_val_id['label'], test_size=0.111, stratify=train_val_id['label'], random_state=42)


train_df=pd.DataFrame()
#for day, egg in zip(train_id['day'], train_id['egg']):
#    train_df=train_df.append(df_full[ (df_full['day']==day) & (df_full['egg']==egg)])
for name in train_id['name']:
    train_df=train_df.append(df_full[ (df_full['name']==name) ])

In [6]:
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Train set size: {len(test_df)}")

Train set size: 24340
Validation set size: 3040
Train set size: 3043


In [7]:
dst_base_PATH='../Dataset/preprocessed/classification/'
folder_list=['train','val','test']

for fd in folder_list:
    folder_path=osp.join(dst_base_PATH, fd)
    if not osp.exists(folder_path):
        os.mkdir(folder_path)
        
    for i in range(2):
        folder=osp.join(folder_path, str(i))
        if not osp.exists(folder):
            os.mkdir(folder)

In [8]:
dst_base_PATH='../Dataset/preprocessed/classification'

df_dict={'train': train_df, 'val':val_df, 'test': test_df}

for key in df_dict:    
    for idx in tqdm(range(len(df_dict[key]))):
        label=str(int(df_dict[key].iloc[idx]['label']))
        img_path=df_dict[key].iloc[idx]['image_path']
        json_path=df_dict[key].iloc[idx]['json_path']
        
        img_name=os.path.split(img_path)[1]
        json_name=os.path.split(json_path)[1]

        # image copy
        img_dst_path=os.path.join(dst_base_PATH, key, label, img_name)
        _=shutil.copyfile(img_path, img_dst_path)
        
        # json copy
        json_dst_path=os.path.join(dst_base_PATH, key, label, json_name)
        _=shutil.copyfile(json_path, json_dst_path)        

100%|██████████| 3043/3043 [00:03<00:00, 971.15it/s] 
